In [ ]:
import pandas
import sklearn
import os 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score

os.chdir('../../../../..')
os.chdir('/Users/shrin/Documents')
data = pd.read_csv('kddcup.data_10_percent_corrected')

# Display the number of rows and columns
num_rows, num_columns = data.shape

# Print the number of rows
print("Number of Rows:", num_rows)

# Set offsets for the fields I am interested in
label_column_offset = 41
last_column_offset = -1 

# Features
X = data.iloc[:, :label_column_offset]  # Use offsets for features
# Labels
y = data.iloc[:, label_column_offset]  # Use offset for labels

# Define a function to categorize 'Probe' and 'Non-Probe'
def categorize_class(value):
    probe_values = ['ipsweep', 'nmap', 'portsweep', 'satan']
    if value.lower() in probe_values:
        return 'Probe'
    else:
        return 'Non-Probe'


# Apply the categorization function to the labels
y = data.iloc[:, last_column_offset].apply(categorize_class)

# Define column offsets for one-hot encoding
categorical_offsets = [1, 2, 5]  # Assuming these are the offsets of 'protocol_type', 'service', and 'flag'
# One-hot encode categorical features
X = pd.get_dummies(X, columns=X.columns[categorical_offsets])

# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create a RandomUnderSampler instance
under_sampler = RandomUnderSampler(sampling_strategy='auto', random_state=42)

# Apply the fit_resample method to obtain the resampled training data
X_resampled, y_resampled = under_sampler.fit_resample(X_train_scaled, y_train)

# Now X_resampled and y_resampled contain the under-sampled data
# Train your model on X_resampled and y_resampled

# Example: Train a Logistic Regression model
model = LogisticRegression()
model.fit(X_resampled, y_resampled)

# Make predictions on the original test set
y_pred = model.predict(X_test_scaled)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_report_output = classification_report(y_test, y_pred)

# Display the results
print(f"Accuracy: {accuracy:.2f}")
print("\nClassification Report:")
print(classification_report_output)

Number of Rows: 494020
